In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil
import json
import keras
import itertools
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import VGG19
from keras.models import Model
import tensorflow as tf
from sklearn.metrics import confusion_matrix

2023-02-22 23:06:27.384150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 23:06:30.191215: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/prp/anaconda3/lib/
2023-02-22 23:06:30.191297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/prp/anaconda3/lib/
2023-02-22 23:06:30.191304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c

In [3]:
main_dir = '/mnt/d/Learning/tensorflow_learning/datasets/cassava_leaf/'
os.listdir(main_dir) 
train_img_path = '/mnt/d/Learning/tensorflow_learning/datasets/cassava_leaf/train_images'

In [4]:
data = pd.read_csv(main_dir+'/train.csv')
data.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [5]:
js = open(main_dir + 'label_num_to_disease_map.json')
real_classes = json.load(js)
real_classes = {int(k):v for k,v in real_classes.items()}

data['class_name'] = data.label.map(real_classes)

In [6]:
data

,image_id,label,class_name
0,1000015157.jpg,0,Cassava Bacterial Blight (CBB)
1,1000201771.jpg,3,Cassava Mosaic Disease (CMD)
2,100042118.jpg,1,Cassava Brown Streak Disease (CBSD)
3,1000723321.jpg,1,Cassava Brown Streak Disease (CBSD)
4,1000812911.jpg,3,Cassava Mosaic Disease (CMD)
...,...,...,...
21392,999068805.jpg,3,Cassava Mosaic Disease (CMD)
21393,999329392.jpg,3,Cassava Mosaic Disease (CMD)
21394,999474432.jpg,1,Cassava Brown Streak Disease (CBSD)
21395,999616605.jpg,4,Healthy


In [7]:
train,val = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['class_name'])

In [36]:
img_row= 400
img_col=400

train_datagen = ImageDataGenerator(rescale = 1/255.0,
                            fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_dataframe(train,
                                                directory = train_img_path,
                                                x_col = 'image_id',
                                                y_col = 'class_name',
                                                target_size = (img_row,img_col),
                                                color_mode = 'rgb',
                                                class_mode = 'categorical',
                                                interpolation = 'nearest',
                                                shuffle = True,
                                                batch_size = 64, 
                                                )


validation_generator = validation_datagen.flow_from_dataframe(val,
                                                directory = train_img_path,
                                                x_col = 'image_id',
                                                y_col = 'class_name',
                                                target_size = (img_row,img_col),
                                                color_mode = 'rgb',
                                                class_mode = 'categorical',
                                                interpolation = 'nearest',
                                                shuffle = True,
                                                batch_size = 64, 
                                                )

Found 19257 validated image filenames belonging to 5 classes.
Found 2140 validated image filenames belonging to 5 classes.


In [37]:
from keras.applications import VGG19
vgg19 = VGG19(weights = 'imagenet',
             include_top = False,
             input_shape = (img_row, img_col,3))

In [38]:
for layer in vgg19.layers:
    layer.trainable = False

In [16]:
def topmodel(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = tf.keras.layers.Flatten(name='flatten')(top_model)
    top_model= tf.keras.layers.Dense(num_classes, activation='softmax')(top_model)
    return top_model

In [39]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu', input_shape=(400, 400, 3)),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding='valid'),

    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding='valid'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
], name= "Small_vgg_model_1")

In [40]:
model_1.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [41]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 mode='min',
                                                 patience=2,
                                                 restore_best_weights=True,
                                                 verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint("Cassava_model.h5",
                                    save_best_only=True,
                                    monitor = 'val_loss',
                                    mode='min')

In [42]:
history = model_1.fit(train_generator,
                   epochs = 3,
                   steps_per_epoch= 301,
                   validation_data = validation_generator,
                   validation_steps =34 ,
                   callbacks = [earlystop,checkpoint],
                   )

Epoch 1/3
301/301 [==============================] - 345s 1s/step - loss: 1.1216 - accuracy: 0.6146 - val_loss: 1.0711 - val_accuracy: 0.6243
Epoch 2/3
301/301 [==============================] - 324s 1s/step - loss: 0.9411 - accuracy: 0.6490 - val_loss: 1.0229 - val_accuracy: 0.6402
Epoch 3/3
301/301 [==============================] - 310s 1s/step - loss: 0.6562 - accuracy: 0.7576 - val_loss: 1.3389 - val_accuracy: 0.6262
